In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 모델 로드

In [2]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

#### 요약문 생성

In [4]:
from torch.utils.data import DataLoader
import pandas as pd

In [5]:
# 테스트 데이터셋 로드
df = pd.read_csv("/content/drive/MyDrive/응용자연어처리/project/data/long_test.csv")

In [6]:
# 배치 요약문 생성 함수
def generate_batch_summaries(model, tokenizer, texts, device, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    summaries = []
    for batch in dataloader:
        # 텍스트를 배치로 토큰화
        inputs = tokenizer(
            list(batch),
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding="max_length"
        ).to(device)

        # 모델로 요약 생성
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 요약문 디코딩
        batch_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(batch_summaries)
    return summaries

# GPU/CPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 배치 처리 및 요약문 생성
batch_size = 16
df["generated_summary"] = generate_batch_summaries(
    model, tokenizer, df["contents"].tolist(), device, batch_size
)

In [7]:
df.head()

,contents,summary,char_length,length,generated_summary
0,"122개 公기관, 지방 추가이전 시급 혁신도시 활성화 대책도 마련해야 지역균형발전은...",지역균형발전은 분권과 함께 노무현정부의 화두였지만 결론적으로는 수도권 집중이 심화되...,1765,long,‘권역별 GRDP 증감 현황’을 분석한 결과 2017년 수도권 GRDP는 773조8...
1,5일 오전 9시쯤 대구 동산병원 장례식장 백합관 지하 1층 분향실 복도. 지나다니는...,독도 소방헬기 추락사고 희생자인 중앙119구조본부 소방공무원과 소방항공대원들의 빈소...,1520,long,독도 소방헬기 추락사고 희생자들의 빈소가 아직 마련되지 않아 유족들의 대기실로 쓰이...
2,정영식 사진=최종훈 인스타그램 캡처 음주운전과 사건무마 청탁 의혹이 불거진 밴드 F...,"14일 소속사 FNC엔터테인먼트는 ""최근 잇따른 사건에 연루돼 물의를 빚은 최종훈에...",1740,long,음주운전과 사건무마 청탁 의혹이 불거진 밴드 FT아일랜드의 최종훈(29)이 연예계 ...
3,"李시장 “4차 산업혁명, 日 뛰어넘는 기회…‘AI 추진단’ 발족” 金지사 “아베정부...",김영록 전남지사가 간부회의에서 4차 산업혁명은 일본을 뛰어넘을 수 있는 절호의 기회...,1568,long,이용섭 시장은 6일 간부회의에서 “4차 산업혁명은 일본을 뛰어넘을 수 있는 절호의 ...
4,공정거래위원회가 내년 1분기에 정보통신기술(ICT) 전담팀 내에 반도체분과를 신설한...,조성욱 공정거래위원장은 19일 세종정부청사에서 열린 취임 100일 기자간담회에서 4...,1728,long,"조성욱 공정거래위원장은 19일 세종정부청사에서 열린 취임 100일 기자간담회에서 ""..."


In [8]:
summaries = df["summary"]
generated_summaries = df["generated_summary"]

#### 성능평가 - long

ROUGE: https://github.com/HeegyuKim/korouge

In [9]:
!pip install korouge_score

In [10]:
import numpy as np
from korouge_score import rouge_scorer

In [11]:
# 한국어 ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

In [12]:
# ROUGE 점수 계산
rouge_scores = []
for reference, generated in zip(df["summary"], df["generated_summary"]):
    scores = scorer.score(reference, generated)
    rouge_scores.append(scores)

# 결과 확인
for i, scores in enumerate(rouge_scores[:5]):
    print(f"Sample {i + 1}:")
    print(scores)
    print()

Sample 1:
{'rouge1': Score(precision=0.0625, recall=0.05263157894736842, fmeasure=0.05714285714285714), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0625, recall=0.05263157894736842, fmeasure=0.05714285714285714), 'rougeLsum': Score(precision=0.0625, recall=0.05555555555555555, fmeasure=0.058823529411764705)}

Sample 2:
{'rouge1': Score(precision=0.35, recall=0.20588235294117646, fmeasure=0.25925925925925924), 'rouge2': Score(precision=0.10526315789473684, recall=0.06060606060606061, fmeasure=0.07692307692307693), 'rougeL': Score(precision=0.25, recall=0.14705882352941177, fmeasure=0.18518518518518517), 'rougeLsum': Score(precision=0.25, recall=0.14705882352941177, fmeasure=0.18518518518518517)}

Sample 3:
{'rouge1': Score(precision=0.5833333333333334, recall=0.16666666666666666, fmeasure=0.25925925925925924), 'rouge2': Score(precision=0.5454545454545454, recall=0.14634146341463414, fmeasure=0.23076923076923073), 'rougeL': Score(precision=0.5833

In [13]:
# ROUGE 점수 평균 계산 (fmeasure 평균)
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
average_scores = {metric: 0 for metric in metrics}

# 총 샘플 수
n = len(rouge_scores)

# 각 메트릭의 fmeasure 합산
for score in rouge_scores:
    for metric in metrics:
        average_scores[metric] += getattr(score[metric], "fmeasure", 0)

# 각 메트릭의 평균 계산
for metric in metrics:
    average_scores[metric] /= n

print(average_scores)

{'rouge1': 0.2335777679613476, 'rouge2': 0.1371405083637271, 'rougeL': 0.21933619742546465, 'rougeLsum': 0.21945724859366086}


BERT-Score: https://github.com/Tiiiger/bert_score

In [14]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [15]:
# BERT-Score
bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
bert_model = BertModel.from_pretrained("beomi/kcbert-base").eval()

def get_bert_embeddings(sentence, tokenizer, model):
    # 문장을 토큰화하고 BERT 입력 형식으로 변환
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # BERT 모델을 통해 문장의 임베딩 얻기
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)

    return embeddings.numpy()

def calculate_bert_score(sentence1, sentence2, tokenizer, model):
    # 각 문장의 BERT 임베딩 벡터 얻기
    embedding1 = get_bert_embeddings(sentence1, tokenizer, model)
    embedding2 = get_bert_embeddings(sentence2, tokenizer, model)

    # 코사인 유사도 계산 (1 - cosine distance)
    score = 1 - cosine(embedding1, embedding2)

    return score

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [16]:
# BERT-Score 점수 계산
df["bert_score"] = df.apply(lambda row: calculate_bert_score(row["summary"], row["generated_summary"], bert_tokenizer, bert_model), axis=1)
print(df.head())

                                            contents  \
0  122개 公기관, 지방 추가이전 시급 혁신도시 활성화 대책도 마련해야 지역균형발전은...   
1  5일 오전 9시쯤 대구 동산병원 장례식장 백합관 지하 1층 분향실 복도. 지나다니는...   
2  정영식 사진=최종훈 인스타그램 캡처 음주운전과 사건무마 청탁 의혹이 불거진 밴드 F...   
3  李시장 “4차 산업혁명, 日 뛰어넘는 기회…‘AI 추진단’ 발족” 金지사 “아베정부...   
4  공정거래위원회가 내년 1분기에 정보통신기술(ICT) 전담팀 내에 반도체분과를 신설한...   

                                             summary  char_length length  \
0  지역균형발전은 분권과 함께 노무현정부의 화두였지만 결론적으로는 수도권 집중이 심화되...         1765   long   
1  독도 소방헬기 추락사고 희생자인 중앙119구조본부 소방공무원과 소방항공대원들의 빈소...         1520   long   
2  14일 소속사 FNC엔터테인먼트는 "최근 잇따른 사건에 연루돼 물의를 빚은 최종훈에...         1740   long   
3  김영록 전남지사가 간부회의에서 4차 산업혁명은 일본을 뛰어넘을 수 있는 절호의 기회...         1568   long   
4  조성욱 공정거래위원장은 19일 세종정부청사에서 열린 취임 100일 기자간담회에서 4...         1728   long   

                                   generated_summary  bert_score  
0  ‘권역별 GRDP 증감 현황’을 분석한 결과 2017년 수도권 GRDP는 773조8...    0.710612  
1  독도 소방헬기 추락사고 희생자들의 빈소가 아직 마련되지 않아 유족들의 대기실로 쓰이...    0.771690  
2  음주

In [17]:
# BERT-Score 평균
df["bert_score"].mean()

0.8211204328986028